# Using Weights & Biases Weave with AWS Bedrock

In this notebook you will learn to use our newly realeased tool for the LLM practitioner

You can use [Weave](https://wandb.github.io/weave/) to:

- Log and debug language model inputs, outputs, and traces
- Build rigorous, apples-to-apples evaluations for language model use cases
- Organize all the information generated across the LLM workflow, from experimentation to evaluations to production

## Setup

In [1]:
!pip install -Uq wandb weave


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import json
import boto3
from botocore.exceptions import ClientError

## log to your W&B account

In [ ]:
import wandb
wandb.login()

## create a W&B project to store your results

In [5]:
weave.init('bedrock-weave')

Logged in as W&B user capecape.
View Weave data at https://wandb.ai/capecape/bedrock-weave/weave


Decorate your function call, that's it!

In [15]:
@weave.op() # <- just add this 😎
def generate_text(
    model_id: str, 
    prompt: str, 
    max_tokens: int=400,
    temperature: float=0.7,
    top_p: float=0.7,
    top_k: int=50,
) -> dict:
    bedrock = boto3.client(service_name='bedrock-runtime')


    body = json.dumps({
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
    })
    
    response = bedrock.invoke_model(
        body=body,
        modelId=model_id
    )

    response_body = json.loads(response.get('body').read())

    outputs = response_body.get('outputs')
    
    return outputs

Let's first try using Mistral 7B

In [18]:
model_id = 'mistral.mistral-7b-instruct-v0:2'

prompt = """<s>[INST] Create a Vegan Carbonara Recipe[/INST]"""

outputs = generate_text(model_id, prompt)

for index, output in enumerate(outputs):

    print(f"Output {index + 1}\n----------")
    print(f"Text:\n{output['text']}\n")
    print(f"Stop reason: {output['stop_reason']}\n")

🍩 https://wandb.ai/capecape/bedrock-weave/r/call/5640fce1-e154-4d1e-80f4-82dda0050d0b
Output 1
----------
Text:
 Title: Vegan Carbonara with Creamy Cashew Sauce

Prep Time: 15 minutes
Cook Time: 10 minutes
Total Time: 25 minutes

Servings: 4

Ingredients:
- 12 oz (340g) spaghetti or your preferred pasta
- 1 1/2 cups (375ml) unsweetened almond or soy milk
- 1 1/2 cups (180g) raw cashews, soaked in water for at least 2 hours
- 1/2 cup (120ml) vegetable broth
- 1/4 cup (60g) nutritional yeast
- 2 tbsp (30ml) olive oil, divided
- 1 tbsp (15ml) lemon juice
- 1 tbsp (15ml) apple cider vinegar
- 1 tsp (5ml) garlic powder
- 1 tsp (5ml) onion powder
- 1 tsp (5ml) smoked paprika (optional)
- 1 tsp (5ml) salt, or to taste
- 1/2 tsp (2.5ml) black pepper, or to taste
- 1/2 cup (120g) cooked crumbled tofu or vegan bacon (optional)
- 1/2 cup (120g) chopped fresh parsley
- 1/4 cup (60g) chopped scallions

Instructions:

1. Cook pasta according to package instructions. Drain and set aside, reserving 1 

## Translation project

Let's use these powerful LLMs to translate the documentation of Weights & Biases

In [31]:
from pathlib import Path
from dataclasses import dataclass

system_prompt = """
# Instructions

You are a documentation translation assistant from English to {output_language}. We are translating valid docusaurus flavored markdown. Some rules to remember:

- Do not add extra blank lines.
- The results must be valid docusaurus markdown
- It is important to maintain the accuracy of the contents but we don't want the output to read like it's been translated. So instead of translating word by word, prioritize naturalness and ease of communication.
- In code blocks, just translate the comments and leave the code as is.


## Formatting Rules

Do not translate target markdown links. Never translate the part of the link inside (). For instance here [https://wandb.ai/site](https://wandb.ai/site) do not translate anything, but on this, you should translate the [] part:
[track metrics](./guides/track), [create logs](./guides/artifacts).
"""
human_prompt = """
Here is a chunk of documentation in docusaurus Markdown format to translate. Return the translation only, without adding anything else. 
<Markdown start>
{md_chunk}
<End of Markdown>
"""

@dataclass
class PromptTemplate:
    system_prompt: str
    human_prompt: str
    language: str
    
    def format_mistral_instruct(self, md_chunk):
        "A formatting function for Mistral Instruct models"
        system_prompt = self.system_prompt.format(output_language=self.language)
        human_prompt = self.human_prompt.format(md_chunk=md_chunk)
        prompt = f"<s>[INST] {system_prompt}\n{human_prompt}[/INST]"""
        return prompt

In [32]:
prompt_template = PromptTemplate(system_prompt, human_prompt, "spanish")

let's read 2 files from our documentation

In [33]:
docs = [p.read_text() for p in Path("./docs").iterdir() if p.suffix == ".md"]
len(docs)

2

We can now call the model with this new prompt:

In [40]:
for doc in docs:
    output = generate_text(model_id, prompt_template.format_mistral_instruct(doc), max_tokens=2048)
    print(f"Text:\n{output[0]['text']}\n")
    print(f"Stop reason: {output[0]['stop_reason']}\n")

🍩 https://wandb.ai/capecape/bedrock-weave/r/call/7d6a04b0-479b-43d7-a7af-63301407d647
Text:
 ---
description: Inicio rápido de W&B.
displayed_sidebar: default
---

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';

# Inicio rápido

Instale W&B y monitoree tus experimentos de aprendizaje automático en minutos.

## 1. Crea una cuenta y Instala W&B
Antes de empezar, asegúrate de crear una cuenta y de instalar W&B:

1. [Regístrate](https://wandb.ai/site) para una cuenta gratuita en [https://wandb.ai/site](https://wandb.ai/site) y luego inicia sesión en tu cuenta de wandb.  
2. Instala la biblioteca wandb en tu máquina en un entorno Python 3 usando [`pip`](https://pypi.org/project/wandb/).  
<!-- 3. Inicia sesión en la biblioteca de wandb en tu máquina. Encontrarás tu clave API aquí: [https://wandb.ai/authorize](https://wandb.ai/authorize).   -->

Los siguientes fragmentos de código muestran cómo instalar y iniciar sesión en W&B usando la CLI y la Biblioteca de Python de

We can improve our weave experience by creating a `Model`

In [42]:
from weave import Model

In [53]:
class MistralInstruct(Model):
    prompt_template: PromptTemplate
    temperature: float=0.7
    max_tokens: int=2048
    model_id: str='mistral.mistral-7b-instruct-v0:2'

    @weave.op()
    def predict(self, document: str) -> dict:
        output = generate_text(
             model_id, 
             self.prompt_template.format_mistral_instruct(document), 
             temperature=self.temperature,
             max_tokens=self.max_tokens,
        )
        return output[0]['text']

In [56]:
model = MistralInstruct(prompt_template=prompt_template)

In [57]:
model.predict(docs[0])

🍩 https://wandb.ai/capecape/bedrock-weave/r/call/1b4327fc-4398-46f6-9522-bffae3af0a7f


' ---\ndescription: Inicio rápido de W&B.\ndisplayed_sidebar: default\n---\n\nimport Tabs from \'@theme/Tabs\';\nimport TabItem from \'@theme/TabItem\';\n\n# Inicio rápido\n\nComience a rastrear sus experimentos de aprendizaje automático en W&B en minutos.\n\n## 1. Crea una cuenta y instala W&B\nAntes de empezar, cree una cuenta y instale W&B:\n\n1. [Regístrate](https://wandb.ai/site) gratis en [https://wandb.ai/site](https://wandb.ai/site) y luego inicie sesión en tu cuenta de wandb.  \n2. Instale la biblioteca wandb en tu máquina en un entorno Python 3 usando [`pip`](https://pypi.org/project/wandb/).  \n<!-- 3. Inicie sesión en la biblioteca wandb en tu máquina. Encuentre su clave API aquí: [https://wandb.ai/authorize](https://wandb.ai/authorize).   -->\n\nLos siguientes fragmentos de código muestran cómo instalar y iniciar sesión en W&B usando la CLI y la biblioteca de Python:\n\n<Tabs\n  defaultValue="notebook"\n  values={[\n    {label: \'Notebook\', value: \'notebook\'},\n    {lab